In [1]:
%pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
downloaded = drive.CreateFile({'id':'1-xlAOqzRfNjVwKVk_iC4m69DGCET8bRu'}) # replace the id with id of file you want to access
downloaded.GetContentFile('nb.geojson') 

In [5]:
downloaded = drive.CreateFile({'id':'1Pk022njkRCgVMNdkJTlctYZWJpp_leOQ'})
downloaded.GetContentFile('sb.geojson') 

In [6]:
downloaded = drive.CreateFile({'id':'1ORFc068YYz4khyxkoPkBGT2YzJSwxO-5'})
downloaded.GetContentFile('ab.geojson') 

In [7]:
downloaded = drive.CreateFile({'id':'1ZJb8MO01Uf4GtM8XZkS_Wpkn3we0PHcz'})
downloaded.GetContentFile('db.geojson') 

In [8]:
downloaded = drive.CreateFile({'id':'1yY48n6vbGggh3pVuJukmYj_F6nNQqoak'})
downloaded.GetContentFile('sb-nb.geojson')

In [9]:
import folium
from folium import plugins
import pandas as pd

import ipywidgets
import os 
import json

from IPython.display import YouTubeVideo
import datetime


---
## Intitialize Map
---

In [10]:
# the latitude and Longitude coordinates
UMMlocation = (10.232066627298536, 76.40871095397823)
map_UMM = folium.Map(location = UMMlocation, width = "75%", zoom_start = 17) # max zoom: 18
map_UMM

In [11]:
hauseOutline = 'nb.geojson'
display(folium.GeoJson(hauseOutline, name="nb").add_to(map_UMM))
hauseOutline = 'sb.geojson'
display(folium.GeoJson(hauseOutline, name="sb").add_to(map_UMM))
hauseOutline = 'ab.geojson'
display(folium.GeoJson(hauseOutline, name="ab").add_to(map_UMM))
hauseOutline = 'db.geojson'
display(folium.GeoJson(hauseOutline, name="db").add_to(map_UMM))
display(map_UMM)

---
##Display Ant Path
---

In [12]:
testGeoJson = 'sb-nb.geojson'

In [13]:
def switchPosition(coordinate):
  temp = coordinate[0]
  coordinate[0] = coordinate[1]
  coordinate[1] = temp
  return coordinate

In [14]:
with open(testGeoJson) as f:
  testWay = json.load(f)

for feature in testWay['features']:
    path = feature['geometry']['coordinates']   
    display(path)
finalPath = list(map(switchPosition,path))
finalPath

[[76.4083081483841, 10.230686182280213],
 [76.4084556698799, 10.230657147079652],
 [76.40848785638809, 10.23083399780556],
 [76.40869706869125, 10.230786485679914],
 [76.40894651412964, 10.232069310576165]]

[[10.230686182280213, 76.4083081483841],
 [10.230657147079652, 76.4084556698799],
 [10.23083399780556, 76.40848785638809],
 [10.230786485679914, 76.40869706869125],
 [10.232069310576165, 76.40894651412964]]

In [15]:
path = 'sb-nb.geojson'
folium.plugins.AntPath([[10.23067034489841, 76.40841007232666],
 [10.230659786643445, 76.40847444534302],
 [10.230828718680836, 76.40850126743317],
 [10.230765369177346, 76.40895187854767],
 [10.232100985199377, 76.40926837921143]]).add_to(map_UMM)
map_UMM

In [16]:
class navigator:
    def __init__(self):
        self.geoResources = {}
        self.buildingLocation =(10.232066627298536, 76.40871095397823)
        self.position = 'sb2'
        self.destination = 'nb'

        for root, dirs, files in os.walk('/content/drive/MyDrive/INDOOR MAPPING'):  
            for file in files:
                self.geoResources[file.split('.')[0]] = root+'/'+file
        display(self.geoResources)
    def changeDestination(self,newDestination):
        #exit if case insert
        if newDestination == 'em':
            if self.position == 'sb2':
                newDestination = 'em1'
            elif self.position == 'nb2':
                newDestination = 'em2'
            elif self.position == 'db2':
                newDestination = 'em3'
            elif self.position == 'ab2':
                newDestination = 'em3'             
        self.destination = newDestination
        self.redrawMap()

    def changeStartPoint(self, newStartPoint):
        
        self.position = newStartPoint #does not work yet
        print(f'Selected Start: {newStartPoint}; Selected Target: {self.destination}')
        #self.redrawMap()
        

    def drawPathWay(self,buildingMap):
      
      def switchPosition(coordinate):
        temp = coordinate[0]
        coordinate[0] = coordinate[1]
        coordinate[1] = temp
        return coordinate

      searchString = self.position + self.destination
      with open(self.geoResources[searchString]) as f:
           testWay = json.load(f)

      for feature in testWay['features']:
        path = feature['geometry']['coordinates']

      finalPath = list(map(switchPosition,path))
      folium.plugins.AntPath(finalPath).add_to(buildingMap)

    def drawBuilding(self,buildingMap):
      hauseOutline = self.geoResources[self.destination]
      folium.GeoJson(hauseOutline, name="geojson").add_to(buildingMap)

    def redrawMap(self):
        #print(f'position {self.position}, destination {self.destination}')
        buildingMap = folium.Map(location = self.buildingLocation, width = "75%", zoom_start = 17)
        self.drawPathWay(buildingMap)
        self.drawBuilding(buildingMap)
        display(buildingMap)




In [17]:
myNavigator = navigator()

{'ab': '/content/drive/MyDrive/INDOOR MAPPING/ab.geojson',
 'ab2ccf': '/content/drive/MyDrive/INDOOR MAPPING/PATH/ab2ccf.geojson',
 'ab2db': '/content/drive/MyDrive/INDOOR MAPPING/PATH/ab2db.geojson',
 'ab2em3': '/content/drive/MyDrive/INDOOR MAPPING/PATH/ab2em3.geojson',
 'ab2gh': '/content/drive/MyDrive/INDOOR MAPPING/PATH/ab2gh.geojson',
 'ab2mba': '/content/drive/MyDrive/INDOOR MAPPING/PATH/ab2mba.geojson',
 'ab2mh': '/content/drive/MyDrive/INDOOR MAPPING/PATH/ab2mh.geojson',
 'ab2ms': '/content/drive/MyDrive/INDOOR MAPPING/PATH/ab2ms.geojson',
 'ab2nb': '/content/drive/MyDrive/INDOOR MAPPING/PATH/ab2nb.geojson',
 'ab2office': '/content/drive/MyDrive/INDOOR MAPPING/PATH/ab2office.geojson',
 'ab2princi': '/content/drive/MyDrive/INDOOR MAPPING/PATH/ab2princi.geojson',
 'ab2sb': '/content/drive/MyDrive/INDOOR MAPPING/PATH/ab2sb.geojson',
 'ccf': '/content/drive/MyDrive/INDOOR MAPPING/ccf.geojson',
 'db': '/content/drive/MyDrive/INDOOR MAPPING/db.geojson',
 'db2ab': '/content/drive/MyD

In [18]:
def displayWay(whereTo):
     myNavigator.changeDestination(whereTo)

In [19]:
def changePosition(whereFrom):
    myNavigator.changeStartPoint(whereFrom)

In [20]:
# Destination Selector
selectHouse_widget=ipywidgets.Select(
    
options=['North Block',
    'MBA Seminar Hall',
    'Main Seminar hall',
    'Administrative Block',
    'Decennial Block',
    'Office',
    'Principal Office',
    'CCF Lab',
    'South Block',
    'Emergency Exit',
    'Girls Hostel',
    'Boys Hostel'],
    value='North Block',
    description='Target',
    disabled=False)

# widget function
def selectHouse(way):
    if way == 'North Block' :
        displayWay('nb' ) 
    if way == 'MBA Seminar Hall':
        displayWay('mba')
    if way == 'Administrative Block':
        displayWay('ab')
    if way == 'Decennial Block':
        displayWay('db')
    if way == 'Office':
        displayWay('office')
    if way == 'Principal Office':
        displayWay('princi')
    if way == 'CCF Lab':
        displayWay('ccf')
    if way == 'South Block':
        displayWay('sb')
    if way == 'Emergency Exit':
        displayWay('em')
    if way == 'Girls Hostel':
        displayWay('gh')
    if way == 'Boys Hostel':
        displayWay('mh')
    if way == 'Main Seminar hall':
        displayWay('ms')         

# Position Selector
selectPosition_widget=ipywidgets.Select(
    options=['South Block', 'North Block', 'Administrative Block','Decennial Block'],
    value='South Block',
    description='Start',
    disabled=False)

def selectPosition(position):
    if position == 'South Block':
        changePosition('sb2')
    if position == 'Administrative Block':
        changePosition('ab2')
    if position == 'North Block':
        changePosition('nb2')
    if position == 'Decennial Block':
        changePosition('db2')    
        
#Initialization   
ipywidgets.interact(selectPosition, position=selectPosition_widget)
ipywidgets.interact(selectHouse, way=selectHouse_widget)

interactive(children=(Select(description='Start', options=('South Block', 'North Block', 'Administrative Block…

interactive(children=(Select(description='Target', options=('North Block', 'MBA Seminar Hall', 'Main Seminar h…

<function __main__.selectHouse>